# Cleavage heterogeneity analysis preparation

**Purpose:** To prepare data for an analysis of the factors mediating cleavage site heterogeneity.


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%run -i notebook_setup.py

In [4]:
from paper_utilities import helpers, motifs, cleavage

In [5]:
PROJECT = "/projects/b1080/eks/polyadenylation/yeast"
OUTDIR  = os.path.join(PROJECT, 'manuscript', 'analysis', 'resources')
os.makedirs(OUTDIR, exist_ok = True)


## Pre-process golden sites from *S. cerevisiae* unclustered data

#### Reload and process prediction data for golden sites

In [6]:
gold_data = pd.read_csv(os.path.join(OUTDIR, "scer_golden_dist5.polya_comprehensive.txt"), sep = "\t")
print(gold_data.shape)

gold_data['label'] = gold_data.apply(lambda row : f'{row.name:06d}:{row.chrom}:{row.start}:{row.strand}', axis = 1)

gold_data['long_sequence'] = gold_data['sequence']
gold_data['sequence'] = gold_data['long_sequence'].apply(lambda x : x[int((len(x)-500)/2):int((len(x)+500)/2)])

gold_data['ctr_cleavage']   = 250
gold_data['readvec']        = gold_data['readvec'].apply(lambda x : np.asarray([float(_) for _ in x.strip("][").split(",")]))
gold_data['observed_norm']  = gold_data['readvec'].apply(lambda x: x / np.sum(x))
gold_data['predicted_norm'] = gold_data['cleavage_norm'].apply(lambda x : np.asarray([float(_) for _ in x.strip("][").split(",")]))

gold_data['observed_entropy']  = gold_data['observed_norm'].apply(lambda x : cleavage.calculate_entropy_from_vector(x))
gold_data['predicted_entropy'] = gold_data['predicted_norm'].apply(lambda x : cleavage.calculate_entropy_from_vector(x))

gold_data['supporting_reads_log1p'] = np.log10(gold_data['supporting_reads'] + 1)
gold_data['window_reads_log1p']     = np.log10(gold_data['window_reads'] + 1)


(11673, 18)


#### Identify the relative position of each site within a gene

In [7]:
gold_sites = {}

for i,row in tqdm.tqdm(gold_data.iterrows()):
    
    rowkey = (row['gene'],row['chrom'],row['strand'])
    
    if not (rowkey in gold_sites):
        gold_sites[rowkey] = [row['start']]
    else:
        gold_sites[rowkey].append(row['start'])
    

11673it [00:01, 9623.46it/s]


In [8]:
gold_sites_labels = {}

for rk,pl in gold_sites.items():
    
    gold_sites_labels[rk] = {}
    
    if (len(pl) == 1):
        gold_sites_labels[rk][pl[0]] = 'single'
        
    else:
        spl = sorted(pl)
        g,c,s = rk
            
        gold_sites_labels[rk][spl[0]] = 'first' if (s == '+') else 'last'
        gold_sites_labels[rk][spl[-1]] = 'last' if (s == '+') else 'first'

        for p in spl[1:-1]:
            gold_sites_labels[rk][p] = 'middle'


In [9]:
gold_data['position'] = gold_data.apply(lambda row : gold_sites_labels.get((row['gene'],row['chrom'],row['strand']), {}).get(row['start'], 'NA'), axis = 1)
print(gold_data['position'].value_counts())


middle    4752
last      2869
first     2869
single    1183
Name: position, dtype: int64


#### Tabulate motif occurrences and their location and frequency

In [10]:
with open(os.path.join(OUTDIR, 'motif_definitions.scer.6mers.distance.pickle'), mode = 'rb') as handle:
    hamming_definitions = pickle.load(handle)


In [11]:
gold_motifs = {
    
    'UA_d0' : {
        'motifs'   : hamming_definitions['priority']['TA/TA-rich_d0'],
        'regions'  : [(-90,-30),(-90,-50),(-50,-30),(-105,-90),(-90,-50),(-50,-30),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45),(-15,15),(-30,30)],
        'dir_pref' : ['max','max','max','max','max','max','max','max','max','max','min','min','min','max','max'],
    },
    'UA_d1' : {
        'motifs'   : hamming_definitions['priority']['TA/TA-rich_d0'] + hamming_definitions['priority']['TA/TA-rich_d1'],
        'regions'  : [(-90,-30),(-90,-50),(-50,-30),(-105,-90),(-90,-50),(-50,-30),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45),(-15,15),(-30,30)],
        'dir_pref' : ['max','max','max','max','max','max','max','max','max','max','min','min','min','max','max'],
    },
    'UA_d2' : {
        'motifs'   : hamming_definitions['priority']['TA/TA-rich_d0'] + hamming_definitions['priority']['TA/TA-rich_d1'] + hamming_definitions['priority']['TA/TA-rich_d2'],
        'regions'  : [(-90,-30),(-90,-50),(-50,-30),(-105,-90),(-90,-50),(-50,-30),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45),(-15,15),(-30,30)],
        'dir_pref' : ['max','max','max','max','max','max','max','max','max','max','min','min','min','max','max'],
    },
    
    'A_d0' : {
        'motifs'   : hamming_definitions['priority']['A-rich_d0'],
        'regions'  : [(-60,-15),(-45,-15),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45),(-15,15),(-30,30)],
        'dir_pref' : ['max','max','max','max','max','max','min','min','min','max','max'],
    },
    'A_d1' : {
        'motifs'   : hamming_definitions['priority']['A-rich_d0'] + hamming_definitions['priority']['A-rich_d1'],
        'regions'  : [(-60,-15),(-45,-15),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45),(-15,15),(-30,30)],
        'dir_pref' : ['max','max','max','max','max','max','min','min','min','max','max'],
    },
    'A_d2' : {
        'motifs'   : hamming_definitions['priority']['A-rich_d0'] + hamming_definitions['priority']['A-rich_d1'] + hamming_definitions['priority']['A-rich_d2'],
        'regions'  : [(-60,-15),(-45,-15),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45),(-15,15),(-30,30)],
        'dir_pref' : ['max','max','max','max','max','max','min','min','min','max','max'],
    },
    
    'U_d0' : {
        'motifs'   : hamming_definitions['priority']['T-rich_d0'],
        'regions'  : [(-15,15),(-30,0),(0,30),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45)],
        'dir_pref' : ['max','max','min','max','max','max','max','min','min','min'],
    },
    'U_d1' : {
        'motifs'   : hamming_definitions['priority']['T-rich_d0'] + hamming_definitions['priority']['T-rich_d1'],
        'regions'  : [(-15,15),(-30,0),(0,30),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45)],
        'dir_pref' : ['max','max','min','max','max','max','max','min','min','min'],
    },
    'U_d2' : {
        'motifs'   : hamming_definitions['priority']['T-rich_d0'] + hamming_definitions['priority']['T-rich_d1'] + hamming_definitions['priority']['T-rich_d2'],
        'regions'  : [(-15,15),(-30,0),(0,30),(-60,-45),(-45,-30),(-30,-15),(-15,0),(0,15),(15,30),(30,45)],
        'dir_pref' : ['max','max','min','max','max','max','max','min','min','min'],
    },
}


In [12]:
for motif_type,motif_dict in gold_motifs.items():
    for (mreg,mdir) in zip(motif_dict['regions'], motif_dict['dir_pref']):

        mlist = motif_dict['motifs']
        mdesc = f'{motif_type}_{mreg[0]}_{mreg[1]}'
        print(f"{motif_type} in {mreg} with direction preference `{mdir}` [N = {len(mlist)}]")

        gold_data[f'idxs_{mdesc}']     = gold_data.apply(lambda row: cleavage.find_motifs_locations(row, 'ctr_cleavage', mreg, mlist, 'equal', False, mdir, over_right_edge = True), axis = 1)
        gold_data[f'idx_{mdesc}']      = gold_data[f'idxs_{mdesc}'].apply(lambda x : cleavage.select_motifs(x, index = mdir, relative = False))
        gold_data[f'count_{mdesc}']    = gold_data.apply(lambda row: cleavage.count_motifs(row['sequence'][max([0,int(row['ctr_cleavage']+mreg[0])]):min([len(row['sequence']),int(row['ctr_cleavage']+mreg[1]+5)])], mlist, method = 'equal', overlapping = False, preference = motif_dict['dir_pref'], count_only = True), axis = 1)
        gold_data[f'multiple_{mdesc}'] = (gold_data[f'count_{mdesc}'] > 1)


UA_d0 in (-90, -30) with direction preference `max` [N = 2]
UA_d0 in (-90, -50) with direction preference `max` [N = 2]
UA_d0 in (-50, -30) with direction preference `max` [N = 2]
UA_d0 in (-105, -90) with direction preference `max` [N = 2]
UA_d0 in (-90, -50) with direction preference `max` [N = 2]
UA_d0 in (-50, -30) with direction preference `max` [N = 2]
UA_d0 in (-60, -45) with direction preference `max` [N = 2]
UA_d0 in (-45, -30) with direction preference `max` [N = 2]
UA_d0 in (-30, -15) with direction preference `max` [N = 2]
UA_d0 in (-15, 0) with direction preference `max` [N = 2]
UA_d0 in (0, 15) with direction preference `min` [N = 2]
UA_d0 in (15, 30) with direction preference `min` [N = 2]
UA_d0 in (30, 45) with direction preference `min` [N = 2]
UA_d0 in (-15, 15) with direction preference `max` [N = 2]
UA_d0 in (-30, 30) with direction preference `max` [N = 2]
UA_d1 in (-90, -30) with direction preference `max` [N = 38]
UA_d1 in (-90, -50) with direction preference `ma

#### Quantify the nucleotide density surrounding the max cleavage site

In [13]:
gold_data['nd_A'] = gold_data.apply(lambda row : row['sequence'][int(row['ctr_cleavage']-15):int(row['ctr_cleavage']+15)].count("A") / 30 * 100, axis = 1)
gold_data['nd_C'] = gold_data.apply(lambda row : row['sequence'][int(row['ctr_cleavage']-15):int(row['ctr_cleavage']+15)].count("C") / 30 * 100, axis = 1)
gold_data['nd_G'] = gold_data.apply(lambda row : row['sequence'][int(row['ctr_cleavage']-15):int(row['ctr_cleavage']+15)].count("G") / 30 * 100, axis = 1)
gold_data['nd_T'] = gold_data.apply(lambda row : row['sequence'][int(row['ctr_cleavage']-15):int(row['ctr_cleavage']+15)].count("T") / 30 * 100, axis = 1)

gold_data['nd_T_up'] = gold_data.apply(lambda row : row['sequence'][int(row['ctr_cleavage']-15):int(row['ctr_cleavage'])].count("T") / 15 * 100, axis = 1)
gold_data['nd_T_dn'] = gold_data.apply(lambda row : row['sequence'][int(row['ctr_cleavage']):int(row['ctr_cleavage']+15)].count("T") / 15 * 100, axis = 1)


#### Identify strongly positive golden sites with high classification confidence

In [14]:
gold_tpdata = gold_data.loc[gold_data['classification'] >= 0.9].copy()
print("Golden sites:\t", gold_data.shape)
print("TP golden sites:", gold_tpdata.shape)


Golden sites:	 (11673, 442)
TP golden sites: (11400, 442)


#### Calculate specific motif distances for UA/UA-rich repeats

In [15]:
gold_tpdata['first_UA_d2_-90_-30']      = gold_tpdata['idxs_UA_d2_-90_-30'].apply(lambda x : cleavage.select_motifs(x, 'first'))
gold_tpdata['last_UA_d2_-90_-30']       = gold_tpdata['idxs_UA_d2_-90_-30'].apply(lambda x : cleavage.select_motifs(x, 'last'))
gold_tpdata['secondlast_UA_d2_-90_-30'] = gold_tpdata['idxs_UA_d2_-90_-30'].apply(lambda x : cleavage.select_motifs(x, 'second_last'))

gold_tpdata['dist_first_UA_d2_-90_-30_CS']      = np.abs(gold_tpdata['ctr_cleavage'] - gold_tpdata['first_UA_d2_-90_-30']) - 6
gold_tpdata['dist_last_UA_d2_-90_-30_CS']       = np.abs(gold_tpdata['ctr_cleavage'] - gold_tpdata['last_UA_d2_-90_-30']) - 6
gold_tpdata['dist_secondlast_UA_d2_-90_-30_CS'] = np.abs(gold_tpdata['ctr_cleavage'] - gold_tpdata['secondlast_UA_d2_-90_-30']) - 6


In [16]:
gold_tpdata['first_UA_d1_-90_-30']      = gold_tpdata['idxs_UA_d1_-90_-30'].apply(lambda x : cleavage.select_motifs(x, 'first'))
gold_tpdata['last_UA_d1_-90_-30']       = gold_tpdata['idxs_UA_d1_-90_-30'].apply(lambda x : cleavage.select_motifs(x, 'last'))
gold_tpdata['secondlast_UA_d1_-90_-30'] = gold_tpdata['idxs_UA_d1_-90_-30'].apply(lambda x : cleavage.select_motifs(x, 'second_last'))

gold_tpdata['dist_first_UA_d1_-90_-30_CS']      = np.abs(gold_tpdata['ctr_cleavage'] - gold_tpdata['first_UA_d1_-90_-30']) - 6
gold_tpdata['dist_last_UA_d1_-90_-30_CS']       = np.abs(gold_tpdata['ctr_cleavage'] - gold_tpdata['last_UA_d1_-90_-30']) - 6
gold_tpdata['dist_secondlast_UA_d1_-90_-30_CS'] = np.abs(gold_tpdata['ctr_cleavage'] - gold_tpdata['secondlast_UA_d1_-90_-30']) - 6


#### Confirm that motif distances were calculated correctly - expect an empty data frame

In [17]:
cond1 = ~gold_tpdata['dist_first_UA_d1_-90_-30_CS'].isna() & ~gold_tpdata['dist_first_UA_d1_-90_-30_CS'].between(25,90)
cond2 = ~gold_tpdata['dist_secondlast_UA_d1_-90_-30_CS'].isna() & ~gold_tpdata['dist_secondlast_UA_d1_-90_-30_CS'].between(25,90)
cond3 = ~gold_tpdata['dist_last_UA_d1_-90_-30_CS'].isna() & ~gold_tpdata['dist_last_UA_d1_-90_-30_CS'].between(25,90)

(gold_tpdata
 .loc[cond1 | cond2 | cond3]
 [['ctr_cleavage','count_UA_d1_-90_-30','idxs_UA_d1_-90_-30','first_UA_d1_-90_-30','last_UA_d1_-90_-30','secondlast_UA_d1_-90_-30']]
)


,ctr_cleavage,count_UA_d1_-90_-30,idxs_UA_d1_-90_-30,first_UA_d1_-90_-30,last_UA_d1_-90_-30,secondlast_UA_d1_-90_-30


#### Split sites into low, middle, and high entropy groups based on observed entropy only

In [18]:
gold_tpdata['observed_entropy_group'], observed_entropy_group_bins = pd.qcut(gold_tpdata['observed_entropy'], q = [0,0.2,0.8,1], labels = range(1,4), retbins=True)
print(gold_tpdata['observed_entropy_group'].value_counts(sort = False))

gold_data['observed_entropy_group'] = pd.cut(gold_data['observed_entropy'], bins=[0, observed_entropy_group_bins[1], observed_entropy_group_bins[2], 4], labels = range(1,4))
print(gold_data['observed_entropy_group'].value_counts(sort = False))

print("\nObserved entropy group cutoffs:", observed_entropy_group_bins)


1    2280
2    6840
3    2280
Name: observed_entropy_group, dtype: int64
1    2367
2    6966
3    2340
Name: observed_entropy_group, dtype: int64

Observed entropy group cutoffs: [0.63497327 2.19106129 2.83239175 3.69158609]


#### Split sites into low, middle, and high entropy groups based on predicted entropy

In [19]:
gold_tpdata['predicted_entropy_group'], predicted_entropy_group_bins = pd.qcut(gold_tpdata['predicted_entropy'], q = [0,0.2,0.8,1], labels = range(1,4), retbins=True)
print(gold_tpdata['predicted_entropy_group'].value_counts(sort = False))

gold_data['predicted_entropy_group'] = pd.cut(gold_data['predicted_entropy'], bins=[0, predicted_entropy_group_bins[1], predicted_entropy_group_bins[2], 4], labels = range(1,4))
print(gold_data['predicted_entropy_group'].value_counts(sort = False))

print("\nPredicted entropy group cutoffs:", predicted_entropy_group_bins)


1    2280
2    6840
3    2280
Name: predicted_entropy_group, dtype: int64
1    2347
2    6989
3    2337
Name: predicted_entropy_group, dtype: int64

Predicted entropy group cutoffs: [0.38364624 1.64297899 2.27988049 2.94592164]


#### Identify sites that are in consensus entropy groups for both observed and predicted entropy

In [20]:
gold_tpdata['consensus_entropy'] = (gold_tpdata['observed_entropy_group'] == gold_tpdata['predicted_entropy_group'])
print(gold_tpdata['consensus_entropy'].value_counts())

gold_data['consensus_entropy'] = (gold_data['observed_entropy_group'] == gold_data['predicted_entropy_group'])
print(gold_data['consensus_entropy'].value_counts())


True     7536
False    3864
Name: consensus_entropy, dtype: int64
True     7702
False    3971
Name: consensus_entropy, dtype: int64


#### Record results and save entropy group labels

In [21]:
with open(os.path.join(OUTDIR, 'gold_data.ctr.pickle'), mode = 'wb') as handle:
    pickle.dump(gold_data, handle)


In [22]:
with open(os.path.join(OUTDIR, 'gold_data.cleavage_heterogeneity.ctr.pickle'), mode = 'wb') as handle:
    pickle.dump(gold_tpdata, handle)


In [23]:
gold_hilo = gold_tpdata.loc[gold_tpdata['observed_entropy_group'].isin([1,3]), ['label','observed_entropy_group']].copy()
gold_dict = dict(zip(gold_hilo['label'], gold_hilo['observed_entropy_group']))

with open(os.path.join(OUTDIR, 'gold_data.entropy_groups.ctr.pickle'), mode = 'wb') as handle:
    pickle.dump(gold_dict, handle)
    
print(len(gold_dict)) ## expecting 4560


4560


#### Save datasets for cleavage mechanism analysis

In [24]:
## Subset sites with a single upstream efficiency element

# High entropy sites

cond0 = (gold_tpdata['observed_entropy_group'] == 3)
cond1 = (gold_tpdata['consensus_entropy'] == True)
cond2 = (gold_tpdata['count_UA_d2_-90_-30'] == 5)

condHIGH = (cond0 & cond1 & cond2)
print("High entropy site selection:", cond0.sum(), cond1.sum(), cond2.sum(), condHIGH.sum())

# Low entropy sites

cond0 = (gold_tpdata['observed_entropy_group'] == 1)
cond1 = (gold_tpdata['consensus_entropy'] == True)

condLOW = (cond0 & cond1)
print("Low entropy site selection:", cond0.sum(), cond1.sum(), condLOW.sum())

# Combine sites selected for analysis

out_data = gold_tpdata.loc[condHIGH | condLOW].copy().rename(columns = {
    'idxs_UA_d2_-90_-30'  : 'idxs_EE', 
    'idx_UA_d2_-90_-30'   : 'idx_EE', 
    'count_UA_d2_-90_-30' : 'count_EE',
    'idxs_A_d2_-45_-15'   : 'idxs_PE',
    'idx_A_d2_-45_-15'    : 'idx_PE',
    'count_A_d2_-45_-15'  : 'count_PE',
}).copy()

print("Sites selected for analysis:", out_data.shape)

with open(os.path.join(OUTDIR, 'gold_tpdata.for_uaua_analysis.d2.ctr.pickle'), mode = 'wb') as handle:
    pickle.dump(out_data, handle)


High entropy site selection: 2280 7536 2717 305
Low entropy site selection: 2280 7536 1355
Sites selected for analysis: (1660, 457)


In [25]:
## Subset sites for cleavage site composition analysis

# High entropy sites

cond0 = (gold_tpdata['observed_entropy_group'] == 3)
cond1 = (gold_tpdata['consensus_entropy'] == True)
cond2 = (gold_tpdata['count_U_d2_-15_15'] == 0)
cond3 = (gold_tpdata['count_UA_d2_-15_0'] > 0)
cond4 = (gold_tpdata['count_UA_d2_0_15'] > 0)

condHIGH = (cond0 & cond1 & cond2 & cond3 & cond4)
print("High entropy site selection:", cond0.sum(), cond1.sum(), cond2.sum(), cond3.sum(), cond4.sum(), condHIGH.sum())

# Low entropy sites

cond0 = (gold_tpdata['observed_entropy_group'] == 1)
cond1 = (gold_tpdata['consensus_entropy'] == True)
cond2 = (gold_tpdata['count_U_d2_-15_0'] > 0)
cond3 = (gold_tpdata['count_U_d2_0_15'] > 0)
cond4 = (gold_tpdata['count_UA_d2_-15_0'] == 0)
cond5 = (gold_tpdata['count_UA_d2_0_15'] == 0)

condLOW = (cond0 & cond1 & cond2 & cond3 & cond4 & cond5)
print("Low entropy site selection :", cond0.sum(), cond1.sum(), cond2.sum(), cond3.sum(), cond4.sum(), cond5.sum(), condLOW.sum())

# Combine sites selected for analysis

out_data = gold_tpdata.loc[condLOW | condHIGH].copy()
print("Sites selected for analysis:", out_data.shape)

with open(os.path.join(OUTDIR, 'gold_tpdata.for_uf_df_analysis.d2.ctr.pickle'), mode = 'wb') as handle:
    pickle.dump(out_data, handle)


High entropy site selection: 2280 7536 2029 7392 7214 182
Low entropy site selection : 2280 7536 7287 6760 4008 4186 222
Sites selected for analysis: (404, 457)
